[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/cap07/cap07-main.ipynb)

# Capítulo 7 - Algoritmos com Modelo


## Configurações Iniciais

In [1]:
from IPython.display import clear_output
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # for saving videos
    !apt-get install ffmpeg

    !pip install gymnasium moviepy
    !pip install optuna

    # clone repository
    !git clone https://github.com/pablo-sampaio/rl_facil
    sys.path.append("/content/rl_facil")

else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )

clear_output()

In [2]:
import random as rand
import gymnasium as gym
import numpy as np

import optuna

from util.experiments import repeated_exec
from util.plot import plot_result, plot_multiple_results
from util.notebook import display_videos_from_path

from util.qtable_helper import evaluate_qtable, record_video_qtable

In [3]:
# basta importar o módulo que o ambiente "RaceTrack-v0" é registrado no gym
import envs

## 1 - Q-Learning

O *Q-Learning* é um algoritmo **livre de modelo**, que vamos comparar com o próximo algoritmo, chamado *Dyna-Q* que é um algoritmo **com modelo**.

Recomendamos acessar  o **código** do Q-Learning no `cap05/qlearning_sarsa.py` para relembrar.

Também vamos executar o Q-Learning, logo abaixo, para comparar com o desempenho do próximo algoritmo.

In [4]:
from cap05.qlearning_sarsa import run_qlearning

In [5]:
# escolha o ambiente descomentando uma das linhas abaixo
#ENV_NAME = "Taxi-v3"
#ENV_NAME = "WindyGrid-v0"
ENV_NAME = "RaceTrack-v0"

LR = 0.3
GAMMA = 0.90
EPSILON = 0.1

#VERBOSE = True

In [ ]:
env = gym.make(ENV_NAME)

if ENV_NAME in ["Taxi-v3", "WindyGrid-v0"]:
    rmax = 10.0
    EPISODES = 700
else:
    rmax = 0.0
    EPISODES = 3_000

rewards1, qtable1 = run_qlearning(env, EPISODES, LR, GAMMA, EPSILON)
print("Últimos resultados: media =", np.mean(rewards1[-20:]), ", desvio padrao =", np.std(rewards1[-20:]))

In [ ]:
# Mostra um gráfico de passos x retornos não descontados acumulados
plot_result(rewards1, rmax, cumulative=False, window=30)

In [ ]:
evaluate_qtable(env, qtable1, 10, verbose=True);

In [ ]:
record_video_qtable(ENV_NAME, qtable1, episodes=3, folder='./videos-qlearn')

In [ ]:
display_videos_from_path('./videos-qlearn', speed=0.5)

## 2 - Dyna-Q

O *Dyna-Q* é um algoritmo **com modelo** que é uma extensão do *Q-Learning*. Compare os códigos para entender o Dyna-Q.

In [11]:
def planning(model, planning_steps, Q, lr, gamma):
    all_s_a = list(model.keys())
    if len(all_s_a) < planning_steps:
        samples = rand.choices(all_s_a, k=planning_steps)
    else:
        samples = rand.sample(all_s_a, k=planning_steps)

    for s, a in samples:
        r, next_s, is_terminal = model[(s,a)]
        if is_terminal:
            V_next_s = 0
        else:
            V_next_s = np.max(Q[next_s])
        delta = (r + gamma * V_next_s) - Q[s,a]
        Q[s,a] = Q[s,a] + lr * delta

In [12]:
# Esta é a política. Neste caso, escolhe uma ação com base nos valores da tabela Q, usando uma estratégia epsilon-greedy,
# dividindo a probabilidade igualmente em caso de empates entre ações de valor máximo.
from util.qtable_helper import epsilon_greedy

In [13]:
# Algoritmo Dyna Q
def run_dyna_q(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1, planning_steps=5, verbose=False):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n

    # inicializa a tabela Q
    Q = np.random.uniform(low=-0.01, high=+0.01, size=(env.observation_space.n, num_actions))

    # inicializa o modelo do ambiente
    model = dict()

    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0

        state, _ = env.reset()

        # executa 1 episódio completo, fazendo atualizações na Q-table
        while not done:

            # escolhe a próxima ação -- usa epsilon-greedy
            action = epsilon_greedy(Q, state, epsilon)

            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated

            if terminated:
                # para estados terminais
                V_next_state = 0
            else:
                # para estados não-terminais -- valor máximo (melhor ação)
                V_next_state = np.max(Q[next_state])

            # atualiza a Q-table / direct RL
            delta = (reward + gamma * V_next_state) - Q[state,action]
            Q[state,action] = Q[state,action] + lr * delta

            # atualiza o modelo
            model[state,action] = (reward, next_state, terminated)

            # planejamento / indirect RL
            planning(model, planning_steps, Q, lr, gamma)

            sum_rewards += reward
            state = next_state

        sum_rewards_per_ep.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso
        if verbose and ((i+1) % 100 == 0):
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

    state = env.reset()
    reward = 0

    return sum_rewards_per_ep, Q

In [ ]:
env = gym.make(ENV_NAME)

if ENV_NAME == 'Taxi-v3':
    rmax = 10.0
    EPISODES = 700
else:
    rmax = 0.0
    EPISODES = 3_000

rewards2, qtable2 = run_dyna_q(env, EPISODES, LR, GAMMA, EPSILON, planning_steps=10, verbose=True)
print("Últimos resultados: media =", np.mean(rewards2[-20:]), ", desvio padrao =", np.std(rewards2[-20:]))

In [ ]:
# Mostra um gráfico de passos x retornos não descontados acumulados
plot_result(rewards2, rmax, cumulative='no', window=30)

In [ ]:
evaluate_qtable(env, qtable2, 10, verbose=True);

In [ ]:
record_video_qtable(ENV_NAME, qtable2, episodes=3, folder='./videos-dynaq')

In [ ]:
display_videos_from_path("./videos-dynaq", speed=0.5)

## 3 - Experimentos Q-Learning x Dyna-Q

Nesta seção, você pode fazer experimentos no `Taxi-v3` ou no `RaceTrack-v0`, como preferir. (Porém, com o `RaceTrack` é mais demorado).

Escolha o ambiente na célula de código abaixo.

Os experimentos têm duas partes:
- Na seção 3.1, vamos otimizar os parâmetros do Q-Learning
- Na seção 3.2, vamos rodar o Q-Learning e o Dyna-Q (ambos com os parâmetros do Q-Learning), para comparar


In [19]:
# Escolha abaixo o ambiente
#ENV_NAME_EXPERIMENT_1 = "Taxi-v3"
#ENV_NAME_EXPERIMENT_1 = "WindyGrid-v0"
ENV_NAME_EXPERIMENT_1 = "RaceTrack-v0"

### 3.1 - Otimizando Parâmetros

Veja o script `cap07/optimize_params.py`, que faz uma otimização do parâmetros do Q-Learning puro usando o `optuna`.

A linha abaixo executa o script como um comando externo e, depois, carrega os resultados.

In [ ]:
# roda o script de otimização
if IN_COLAB:
    !python rl_facil/cap07/optimize_params.py --env {ENV_NAME_EXPERIMENT_1}
else:
    !python optimize_params.py --env {ENV_NAME_EXPERIMENT_1}

#clear_output()

In [ ]:
# carrega os resultados
study = optuna.load_study(storage=f"sqlite:///optuna_cap07.db",
                          study_name=f"qlearning_{ENV_NAME_EXPERIMENT_1}")
clear_output()

print(f"MELHORES PARÂMETROS PARA {ENV_NAME_EXPERIMENT_1} (trial #{study.best_trial.number}):")
print(study.best_params)

qlearn_params_1 = study.best_params

Se não quiser rodar a otimização, você pode descomentar uma das linhas abaixo.

In [22]:
# para o Racetrack
#qlearn_params_1 = {'lr': 0.8167256204339587, 'epsilon': 0.06280337647714501, 'gamma': 0.9619087751023728}

# para o Taxi
#qlearn_params_1 = {'lr': 0.9616271568166056, 'epsilon': 0.013769122856570889, 'gamma': 0.8467211706577995}

# para o WindyGrid-v0
#qlearn_params_1 = {'lr': 0.7754141348857424, 'epsilon': 0.010419303097732811, 'gamma': 0.9124779956696675}

### 3.2 - Experimentos Comparativos

In [32]:
environment = gym.make(ENV_NAME_EXPERIMENT_1)

if ENV_NAME_EXPERIMENT_1 in ["Taxi-v3", "WindyGrid-v0"]:
    EPISODES = 1_000
else:
    EPISODES = 3_000

RUNS = 5
AUTO_LOAD = True

results_1 = []

results_1.append( repeated_exec(RUNS, f"Q-Learning ", run_qlearning, environment, EPISODES, **qlearn_params_1, auto_load=AUTO_LOAD) )
clear_output()

#plan_steps = 1
#results_t.append( repeated_exec(RUNS, f"Dyna-Q ({plan_steps} passo)", run_dyna_q, environment, EPISODES, **qlearn_params_taxi, planning_steps=plan_steps, auto_load=AUTO_LOAD) )
#clear_output()

plan_steps = 10
results_1.append( repeated_exec(RUNS, f"Dyna-Q ({plan_steps} passos)", run_dyna_q, environment, EPISODES, **qlearn_params_1, planning_steps=plan_steps, auto_load=AUTO_LOAD) )
clear_output()


In [ ]:
plot_multiple_results(results_1, cumulative='no', x_log_scale=False, window=100)

In [ ]:
plot_multiple_results(results_1, cumulative='avg', x_log_scale=False)

## 4 - Experimentos com Ambientes Estocásticos

Nesta seção, vamos fazer experimentos similares aos da seção 3, mas focando em ambientes estocásticos como o `FrozenLake`.

Este ambiente tem como diferencial o fato de ser **não-determinístico**:
- uma ação tem 1/3 de chance de dar o resultado correto
- e 1/3 de fazer o agente mover em cada direção perpendicular à desejada.

Isso afeta um pouco os resultados, como veremos.

### 4.1 - Otimizando Parâmetros

Escolha o ambiente, na célula de código abaixo.

In [35]:
#ENV_NAME_EXPERIMENT_2 = "FrozenLake-v1"
ENV_NAME_EXPERIMENT_2 = "WindyGrid-v2" # stochastic version

Em seguida, vamos pode novamente usar o script `cap07/optimize_params` para otimizar os parâmetros do Q-Learning puro.

Mas, dessa vez, vamos usar valores previamente calculados...

In [ ]:
'''# roda o script de otimização
if IN_COLAB:
    !python rl_facil/cap07/optimize_params.py --env {ENV_NAME_EXPERIMENT_2} --episodes_per_trial 300 --trials 120
else:
    pass #!python optimize_params.py --env {ENV_NAME_EXPERIMENT_2} --episodes_per_trial 300 --trials 120

# carrega parâmetros ótimos obtidos com o Optuna
study = optuna.load_study(storage="sqlite:///optuna_cap07.db",
                          study_name=f"qlearning_{ENV_NAME_EXPERIMENT_2}")
#clear_output()

print(f"MELHORES PARÂMETROS PARA {ENV_NAME_EXPERIMENT_2} (trial #{study.best_trial.number}):")
print(study.best_params)

qlearn_params_2 = study.best_params'''

A célula abaixo carrega bons parâmetros previamente obtidos com o `optuna`.

In [37]:
if ENV_NAME_EXPERIMENT_2 == "FrozenLake-v1":
    qlearn_params_2 = {'lr': 0.3692331683709833, 'epsilon': 0.10280622937354741, 'gamma': 0.9792780782708356}
elif ENV_NAME_EXPERIMENT_2 == "WindyGrid-v2":
    qlearn_params_2 = {'lr': 0.4859475184416657, 'epsilon': 0.0793371367164761, 'gamma': 0.8189890984802477}

### 4.2 - Resultados do Dyna-Q sem alteração

In [ ]:
environment = gym.make(ENV_NAME_EXPERIMENT_2)
EPISODES = 2_000 if ENV_NAME_EXPERIMENT_2 == "FrozenLake-v1" else 1_000
RUNS = 15
AUTO_LOAD = False

results_2 = []

results_2.append( repeated_exec(RUNS, f"Q-Learning ", run_qlearning, environment, EPISODES, **qlearn_params_2, auto_load=AUTO_LOAD) )
clear_output()

plan_steps = 5
results_2.append( repeated_exec(RUNS, f"Dyna-Q ({plan_steps} passos)", run_dyna_q, environment, EPISODES, **qlearn_params_2, planning_steps=plan_steps, auto_load=AUTO_LOAD) )
clear_output()

In [ ]:
plot_multiple_results(results_2, cumulative='no', x_log_scale=False, window=100, y_min=-300)

In [40]:
#plot_multiple_results(results_2, cumulative='avg', x_log_scale=False)

Analisando os resultados, o Dyna-Q foi bem nesse ambiente estocástico (não-determinístico)?

Veja que, apesar do desempenho variar entre execuções, ele não costuma ir bem com quantidades altas de "passos de planejamento" neste ambiente.

**O que explica isso?**

### 4.3 - Desafio

Proponha uma modificação simples no **modelo** e no **planejamento** do *Dyna-Q* para melhorar o desempenho dele nos ambiente estocásticos.

Depois, refaça os experimentos da seção 4.2 para conferir se deu certo. (O desempenho deve ser melhor do que o do *Dyna-Q* original, pelo menos.)

In [ ]:
def planning_new(model, planning_steps, Q, lr, gamma):
    pass
    # TODO: PROPONHA UMA ALTERAÇÃO AQUI

raise NotImplementedError("Implemente a função planning_new acima antes de prosseguir")

In [ ]:
# Algoritmo Dyna Q
def run_dyna_q_new(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1, planning_steps=5):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n

   # inicializa a tabela Q
    Q = np.random.uniform(low=-0.01, high=+0.01, size=(env.observation_space.n, num_actions))

    # inicializa o modelo
    model = dict()  # TODO - altere aqui, se precisar

    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0

        state, _ = env.reset()

        # executa 1 episódio completo, fazendo atualizações na Q-table
        while not done:

            # escolhe a próxima ação -- usa epsilon-greedy
            action = epsilon_greedy(Q, state, epsilon)

            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated

            if terminated:
                # para estados terminais
                V_next_state = 0
            else:
                # para estados não-terminais -- valor máximo (melhor ação)
                V_next_state = np.max(Q[next_state])

            # atualiza a Q-table / direct RL
            delta = (reward + gamma * V_next_state) - Q[state,action]
            Q[state,action] = Q[state,action] + lr * delta

            # atualiza o modelo
            # TODO: PROPONHA UMA ALTERAÇÃO AQUI

            # planejamento / indirect RL
            planning_new(model, planning_steps, Q, lr, gamma)

            sum_rewards += reward
            state = next_state

        sum_rewards_per_ep.append(sum_rewards)

    state = env.reset()
    reward = 0

    return sum_rewards_per_ep, Q

### 4.4 - Novos Experimentos

In [ ]:
environment = gym.make(ENV_NAME_EXPERIMENT_2)
results_2b = []

results_2b.append( repeated_exec(RUNS, f"Q-Learning ", run_qlearning, environment, EPISODES, **qlearn_params_2, auto_load=AUTO_LOAD) )
clear_output()

plan_steps = 5
results_2b.append( repeated_exec(RUNS, f"Dyna-Q ({plan_steps} passos)", run_dyna_q, environment, EPISODES, **qlearn_params_2, planning_steps=plan_steps, auto_load=AUTO_LOAD) )
clear_output()

plan_steps = 5
results_2b.append( repeated_exec(RUNS, f"Dyna-Q-new ({plan_steps} passos)", run_dyna_q_new, environment, EPISODES, **qlearn_params_2, planning_steps=plan_steps, auto_load=False) )
clear_output()

In [ ]:
plot_multiple_results(results_2b, cumulative='no', x_log_scale=False, window=100, y_min=-300)